In [219]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from jupyterthemes import jtplot
import os
jtplot.style()
%matplotlib inline

In [220]:
df = pd.read_csv("AMD.csv")
data = df.Open.values
data = data.astype('float32')

In [221]:
train_size = int(len(data) * 0.80)
test_size = len(data) - train_size
train, test = data[0:train_size], data[train_size:len(data)]
train, test = np.reshape(train, (-1, 1)), np.reshape(test, (-1, 1))

In [222]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(train)
train = scaler.transform(X=train)
test = scaler.transform(X=test)

In [223]:
# create an additional column with values at time T=t+1
def create_dataset(dataset, step=1):

    dataX, dataY = [], []
    for i in range(len(dataset) - 1 - step):
        dataX.append(dataset[i])
        dataY.append(dataset[i + step])

    return np.array(dataX), np.array(dataY)

In [224]:
trainX, trainY = create_dataset(train, 1)
testX, testY = create_dataset(test, 1)
trainX = np.reshape(trainX, (-1, 1))
trainY = np.reshape(trainY, (-1,))
testX = np.reshape(testX, (-1, 1))
testY = np.reshape(testY, (-1,))

In [225]:
n_stocks = trainX.shape[1]
# placeholders
X = tf.placeholder(dtype=tf.float32, shape=[None, n_stocks])
Y = tf.placeholder(dtype=tf.float32, shape=[None])

In [226]:
# neurons
n_neurons_1 = 1024
n_neurons_2 = 512
n_neurons_3 = 256
n_neurons_4 = 128
n_target = 1


In [227]:
# Initializers
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode='fan_avg', distribution='uniform', scale=sigma)
bias_initializer = tf.zeros_initializer()

In [228]:
# Layer 1
W_hidden_1 = tf.Variable(weight_initializer([n_stocks, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
# Layer 2
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
# Layer 3
W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
# Layer 4
W_hidden_4 = tf.Variable(weight_initializer([n_neurons_3, n_neurons_4]))
bias_hidden_4 = tf.Variable(bias_initializer([n_neurons_4]))

In [229]:
# Output layer
W_out = tf.Variable(weight_initializer([n_neurons_4, n_target]))
bias_out = tf.Variable(bias_initializer([n_target]))

In [230]:
# Hidden layers
hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
hidden_4 = tf.nn.relu(tf.add(tf.matmul(hidden_3, W_hidden_4), bias_hidden_4))

out = tf.transpose(tf.add(tf.matmul(hidden_4, W_out), bias_out))

In [231]:
# Cost function
mse = tf.reduce_mean(tf.squared_difference(out, Y))

# Optimizer
opt = tf.train.AdamOptimizer().minimize(mse)

# Session
net = tf.Session()
net.run(tf.global_variables_initializer())


In [232]:
# epochs and batch size
epochs = 10
batch_size = 8

In [233]:
mse_train = np.array([])
mse_test = np.array([])
for e in range(epochs):
    # shuffle data
    indices = np.random.permutation(np.arange(len(trainY)))
    trainX = trainX[indices]
    trainY = trainY[indices]

    for i in range(0, len(trainY) // batch_size):
        start = i * batch_size
        batch_x = trainX[start:start + batch_size]
        batch_x = np.reshape(batch_x, (-1, 1))
        batch_y = trainY[start:start + batch_size]
        batch_y = np.reshape(batch_y, (-1,))
        net.run(opt, feed_dict={X: batch_x, Y: batch_y})

        if np.mod(i, 2) == 0:
            mse_train = np.append(mse_train, net.run(mse, feed_dict={X: trainX, Y: trainY}))
            mse_test = np.append(mse_test, net.run(mse, feed_dict={X: testX, Y: testY}))
            pred = net.run(out, feed_dict={X: testX})
            plt.plot(testY, label='testY')
            plt.plot(np.reshape(pred, (-1, 1)), label='prediction')
            plt.legend()
            plt.title('Epoch ' + str(e) + ', Batch ' + str(i))
            if not os.path.exists('plots'):
                os.mkdir('plots')
            file_name = 'plots/epoch_' + str(e) + '_batch_' + str(i) + '.png'
            plt.savefig(file_name)
            plt.clf()


<Figure size 432x288 with 0 Axes>

In [234]:
print('MSE train : ', mse_train[-1])
print('MSE test : ', mse_test[-1])

MSE train :  0.022356826812028885
MSE test :  0.02710242196917534
